In [11]:
!pip install pandas torch transformers sentencepiece

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import pandas as pd
from transformers import pipeline

In [13]:
# --- Persiapan Model Terjemahan ---
print("--- Mempersiapkan Model Terjemahan ---")
print("Memuat model dari Indonesia ke Inggris...")
translator_id_to_en = pipeline("translation", model="Helsinki-NLP/opus-mt-id-en")

print("Memuat model dari Inggris ke Indonesia...")
translator_en_to_id = pipeline("translation", model="Helsinki-NLP/opus-mt-en-id")
print("Model terjemahan berhasil dimuat.")

--- Mempersiapkan Model Terjemahan ---
Memuat model dari Indonesia ke Inggris...


Device set to use cpu


Memuat model dari Inggris ke Indonesia...


Device set to use cpu


Model terjemahan berhasil dimuat.


In [14]:
# --- Membaca Data dan Filter Sentimen Netral ---
input_filename = '../Data/hasil_pelabelan_transjakarta.csv'
try:
    df = pd.read_csv(input_filename)
    df.dropna(subset=['Teks_Stemmed'], inplace=True)
    df = df[df['Teks_Stemmed'].str.strip() != '']
    print(f"\nFile '{input_filename}' berhasil dibaca. Jumlah data awal: {len(df)} baris.")
except FileNotFoundError:
    print(f"Error: File '{input_filename}' tidak ditemukan.")
    exit()

df_netral = df[df['Sentimen'] == 'netral'].copy()
print(f"Ditemukan {len(df_netral)} berita dengan sentimen 'netral' untuk di-augmentasi.")


File '../Data/hasil_pelabelan_transjakarta.csv' berhasil dibaca. Jumlah data awal: 150 baris.
Ditemukan 14 berita dengan sentimen 'netral' untuk di-augmentasi.


In [15]:
# --- Proses Back-Translation ---
print("\nMemulai proses augmentasi (back-translation)...")
augmented_texts = []
for text in df_netral['Teks_Lengkap']: # Kita augmentasi teks asli sebelum di-stemming
    try:
        # Terjemahkan ke Bahasa Inggris
        translated_to_en = translator_id_to_en(text, max_length=5000)[0]['translation_text']
        
        # Terjemahkan kembali ke Bahasa Indonesia
        back_translated_text = translator_en_to_id(translated_to_en, max_length=5000)[0]['translation_text']
        
        augmented_texts.append(back_translated_text)
        print(f"Berhasil meng-augmentasi 1 baris...")
    except Exception as e:
        print(f"Gagal meng-augmentasi teks karena error: {e}. Melewati baris ini.")
        augmented_texts.append(text) # Jika gagal, gunakan teks asli

# Buat DataFrame baru untuk data yang sudah di-augmentasi
df_augmented = df_netral.copy()
df_augmented['Teks_Lengkap'] = augmented_texts
# Tambahkan penanda bahwa ini adalah data augmentasi
df_augmented['is_augmented'] = True
df['is_augmented'] = False # Tandai data asli

Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors



Memulai proses augmentasi (back-translation)...
Gagal meng-augmentasi teks karena error: index out of range in self. Melewati baris ini.
Gagal meng-augmentasi teks karena error: index out of range in self. Melewati baris ini.
Berhasil meng-augmentasi 1 baris...
Berhasil meng-augmentasi 1 baris...
Gagal meng-augmentasi teks karena error: index out of range in self. Melewati baris ini.
Berhasil meng-augmentasi 1 baris...
Gagal meng-augmentasi teks karena error: index out of range in self. Melewati baris ini.
Gagal meng-augmentasi teks karena error: index out of range in self. Melewati baris ini.
Berhasil meng-augmentasi 1 baris...
Berhasil meng-augmentasi 1 baris...
Berhasil meng-augmentasi 1 baris...
Berhasil meng-augmentasi 1 baris...
Gagal meng-augmentasi teks karena error: index out of range in self. Melewati baris ini.
Gagal meng-augmentasi teks karena error: index out of range in self. Melewati baris ini.


In [16]:
# --- Menggabungkan dan Menyimpan Data ---
# Gabungkan data asli dengan data hasil augmentasi
df_final_augmented = pd.concat([df, df_augmented], ignore_index=True)

output_filename = '../Data/hasil_augmentasi_transjakarta.csv'
df_final_augmented.to_csv(output_filename, index=False, encoding='utf-8')

print(f"\n✅ Proses augmentasi selesai.")
print(f"Jumlah data setelah augmentasi: {len(df_final_augmented)} baris.")
print(f"Hasil disimpan di file: {output_filename}")
print("\nDistribusi Sentimen Baru:")
print(df_final_augmented['Sentimen'].value_counts())


✅ Proses augmentasi selesai.
Jumlah data setelah augmentasi: 164 baris.
Hasil disimpan di file: ../Data/hasil_augmentasi_transjakarta.csv

Distribusi Sentimen Baru:
Sentimen
positif    84
negatif    52
netral     28
Name: count, dtype: int64
